In [8]:
from collections import OrderedDict
from torch.optim.lr_scheduler import StepLR
import torch
from torchvision.ops import nms
from torchvision.ops.boxes import box_convert,box_iou
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2 
from torchvision.models.detection import fasterrcnn_resnet50_fpn 
from torchvision.models.detection import ssd300_vgg16, SSD300_VGG16_Weights, ssd
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import os
import cv2
import random
import datetime
torch.manual_seed = 0
import sys
sys.path.append('..')
from conf import *
from inference.inference_helper import preprocess_image, inference_filter_prediction,denormalize_polygon
from ultralytics import YOLO



model_path1 = "../models/Good_FasterRcnn_V1_epoch-6_model.pth"
model_path2 = "../models/Good_FasterRcnn_V2_epoch-4_model.pth"
model_path3 = "../models/Good_SSD_epoch-8_model.pth"
yolo_path4 = "../models/yolov8.pt"

In [2]:

def load_model1(num_classes=NUMBER_OF_CLASSES):
    model = fasterrcnn_resnet50_fpn(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    state_dict= torch.load(model_path1)
    updated_state = {k.replace("module.", ""): v for k,v in state_dict.items()}
    model.load_state_dict(updated_state)
    print(f"MODEL from volume: {model_path1} is loaded successfuly")
    return model

def load_model2(num_classes=NUMBER_OF_CLASSES):
    model = fasterrcnn_resnet50_fpn_v2(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    state_dict= torch.load(model_path2)
    updated_state = {k.replace("module.", ""): v for k,v in state_dict.items()}
    model.load_state_dict(updated_state)
    print(f"MODEL from volume: {model_path2} is loaded successfuly")
    return model


def get_ssd_detection_model(num_classes=NUMBER_OF_CLASSES):
    ssd_model = ssd300_vgg16(weights=False)
    num_anchors = ssd_model.anchor_generator.num_anchors_per_location()
    out_channels = [512,1024,512,256,256,256]
    ssd_model.head = ssd.SSDHead(out_channels, num_anchors, num_classes+1)
    state_dict= torch.load(model_path3)
    updated_state = {k.replace("module.", ""): v for k,v in state_dict.items()}
    ssd_model.load_state_dict(updated_state)
    print(f"MODEL from volume: {model_path3} is loaded successfuly")

    return ssd_model

def load_yolo():
    model = YOLO(yolo_path4)
    return model



def save_test_img(img, target, prefix):
    # img = img.permute(2,0,1).cpu().numpy()  # Convert to (height, width, channels)
    img = img.cpu().numpy()  # Convert to (height, width, channels)

    # img = img.astype('uint8')
    # img = img
    # Draw bounding boxes on the image
    print(target)
    for box, label in zip(target['boxes'], target['labels']):
        x, y, w, h = box.tolist()
        x, y, w, h = int(x), int(y), int(w), int(h)
        # box_color = BOX_COLOR[label.item()]
        print(label)
        box_color = (255,255,255)
        cv2.rectangle(img, (x, y), (w, h), box_color, 2)

    # Save the image with bounding boxes
    if not os.path.exists(os.path.join(os.getcwd(), 'test_output')):
        os.makedirs(os.path.join(os.getcwd(), 'test_output'))
    # cv2.imshow(img)
    img_path = f"./test_output/output_image_{prefix}.png"
    cv2.imwrite(img_path, img)
    return img_path


def train(model, train_loader, optimizer, epoch):
    device = torch.device("cpu")
    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader, 1):
        data = list(image.to(device) for image in data)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        optimizer.zero_grad()
        output = model(data, targets)
        print(f"=====[ epoch {epoch} batch {batch_idx}  output of the model: {output}")

        loss = output["loss_classifier"]
        loss.backward()
        optimizer.step()

def run(model, train_loader):
    epochs = 2
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

    for epoch in range(1, epochs + 1):
        train(model, train_loader, optimizer, epoch)
        scheduler.step()



In [3]:
# model1 = load_model1()
# model2 = load_model2()
# model3 = get_ssd_detection_model()
model4 = load_yolo()

# model1.eval()
# model2.eval()
# model3.eval()
# model4.eval()

original_image_sizes = []

In [17]:
image_dir = "./real_images/complex.jpg"
image_dim = (960,1280)
denormalized_road_roi_polygon = denormalize_polygon(image_dim, ROAD_ROI_POLYGON)

image = cv2.imread(image_dir)
grayscale = preprocess_image(image)
# grayscale = grayscale.unsqueeze(0)
inputs = grayscale
inputs = inputs.permute(0,3,2,1)

# timer_model1 = datetime.datetime.now()
# outputs1 = model1(inputs)
# timer_model1 = datetime.datetime.now() - timer_model1 

# # print(f"outputs1: {outputs1}")
# timer_model2 = datetime.datetime.now()
# outputs2 = model2(inputs)
# timer_model2 = datetime.datetime.now() - timer_model2

# timer_model3 = datetime.datetime.now()
# outputs3 = model3(inputs)
# timer_model3 = datetime.datetime.now() - timer_model3

timer_model4 = datetime.datetime.now()
outputs4 = model4(inputs)
timer_model4 = datetime.datetime.now() - timer_model4

# print(f"outputs1: {outputs1}")
# print("Model1 filtering: ")
# outputs1 = inference_filter_prediction(outputs1)
# print("Model2 filtering: ")
# outputs2 = inference_filter_prediction(outputs2)
# print("Model3 filtering: ")
# outputs3 = inference_filter_prediction(outputs3)
print("Model4 filtering: ")
# outputs4 = inference_filter_prediction(outputs4, normalized_road_roi_polygon=denormalized_road_roi_polygon)
# print(outputs4)
# print(f"Time\nModel1 (Rcnn_V1): {timer_model1}\nModel2 (Rcnn_V2): {timer_model2}\nModel3 (SSD): {timer_model3}")
# Display the results
# outputs4.show()
for r in outputs4:
    print(r.boxes)

# Save the results (optional)

# it = random.randint(0, 1000)
# prefix=f"detections-{it}"
# print(prefix)
# print(outputs3)

# for output1, output2, output3 in zip(outputs1, outputs2,outputs3):
#     save_test_img(torch.clone(image), output1, f"model1_{prefix}")
#     save_test_img(torch.clone(image), output2, f"model2_{prefix}")
#     save_test_img(image, output3, f"model3_{prefix}")






0: 960x1280 4 vehicles, 190.6ms
Speed: 0.0ms preprocess, 190.6ms inference, 3.9ms postprocess per image at shape (1, 3, 960, 1280)
Model4 filtering: 
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([4., 4., 4., 4.])
conf: tensor([0.3592, 0.3114, 0.2985, 0.2932])
data: tensor([[2.4553e+02, 2.8972e+02, 2.7939e+02, 3.1989e+02, 3.5917e-01, 4.0000e+00],
        [1.3890e+02, 3.2939e+01, 2.4867e+02, 9.6170e+01, 3.1142e-01, 4.0000e+00],
        [3.2268e+02, 1.7719e+02, 3.5399e+02, 2.1675e+02, 2.9854e-01, 4.0000e+00],
        [2.6119e+02, 3.2223e+02, 2.9206e+02, 3.4298e+02, 2.9323e-01, 4.0000e+00]])
id: None
is_track: False
orig_shape: (960, 1280)
shape: torch.Size([4, 6])
xywh: tensor([[262.4594, 304.8049,  33.8540,  30.1717],
        [193.7836,  64.5543, 109.7757,  63.2313],
        [338.3377, 196.9695,  31.3095,  39.5687],
        [276.6281, 332.6047,  30.8663,  20.7512]])
xywhn: tensor([[0.2050, 0.3175, 0.0264, 0.0314],
        [0.1514, 0.0672, 0.0858, 0.0659],
      